In [1]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

In [2]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [3]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 35)

In [4]:
# add observation index
data["row_index"] = data.index

In [5]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 36)
test: (39821, 36)
known: (2319611, 36)
unknown: (19918, 36)


In [6]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff', 'hours',
       'is_listened_lag', 'genre_plays', 'genre_skips', 'artist_plays',
       'artist_skips', 'album_plays', 'album_skips', 'song_plays',
       'song_skips', 'user_ratio_flow', 'user_ratio_full', 'genre_ratio',
       'artist_ratio', 'song_ratio', 'platform_name1', 'platform_name2',
       'platform_family1', 'platform_family2', 'row_index'],
      dtype='object')


## 2. MODELING

### 2.1. INITIALIZING

In [7]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([u, c])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
embedding_dense = BatchNormalization()(embedding_dense)

# Layer with numeric features
data_dense = Dense(16, activation = "relu")(data_in)
data_dense = BatchNormalization()(data_dense)

# Constructing the further layers
x = concatenate([embedding_dense, data_dense])
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([user_in, context_in, data_in], output)
model.compile(optimizer = "Adagrad", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [8]:
# run the estimations on training data
model.fit([tr.user_id, tr.context_type, tr_data], tr.is_listened, 
validation_data = ([ts.user_id, ts.context_type, ts_data], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2279790 samples, validate on 39821 samples
Epoch 1/10
2279790/2279790 [==============================] - 92s - loss: 0.5289 - acc: 0.7384 - val_loss: 0.6705 - val_acc: 0.6085
Epoch 2/10
2279790/2279790 [==============================] - 73s - loss: 0.4877 - acc: 0.7652 - val_loss: 0.6584 - val_acc: 0.6084
Epoch 3/10
2279790/2279790 [==============================] - 53s - loss: 0.4823 - acc: 0.7680 - val_loss: 0.6359 - val_acc: 0.6403
Epoch 4/10
2279790/2279790 [==============================] - 68s - loss: 0.4788 - acc: 0.7695 - val_loss: 0.6229 - val_acc: 0.6568
Epoch 5/10
2279790/2279790 [==============================] - 76s - loss: 0.4766 - acc: 0.7705 - val_loss: 0.6086 - val_acc: 0.6719
Epoch 6/10
2279790/2279790 [==============================] - 77s - loss: 0.4747 - acc: 0.7714 - val_loss: 0.6037 - val_acc: 0.6789
Epoch 7/10
2279790/2279790 [==============================] - 75s - loss: 0.4735 - acc: 0.7719 - val_loss: 0.6089 - val_acc: 0.6794
Epoch 8/10
2279790/22797

In [9]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.user_id, ts.context_type, ts_data])
pred.to_csv(path + "pred_valid/keras_newdata_flow_context50.csv", index = False)
pred.head(5)

,row_index,is_listened
2167,2167,0.980167
2168,2168,0.989523
2240,2240,0.989574
4314,4314,0.985285
4321,4321,0.987488


In [10]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.7230561342740176

### 2.3. SECOND STAGE

In [ ]:
# run the estimations on full known data
model.fit([kn.user_id, kn.context_type, kn_data], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 86s - loss: 0.4724 - acc: 0.7722    
Epoch 2/10
2319611/2319611 [==============================] - 86s - loss: 0.4717 - acc: 0.7723    
Epoch 3/10
2319611/2319611 [==============================] - 110s - loss: 0.4716 - acc: 0.7726   
Epoch 4/10
2319611/2319611 [==============================] - 111s - loss: 0.4707 - acc: 0.7729   
Epoch 5/10
2319611/2319611 [==============================] - 109s - loss: 0.4701 - acc: 0.7731   
Epoch 6/10
2319611/2319611 [==============================] - 111s - loss: 0.4699 - acc: 0.7732   
Epoch 7/10
2319611/2319611 [==============================] - 108s - loss: 0.4696 - acc: 0.7735   
Epoch 8/10
2319611/2319611 [==============================] - 109s - loss: 0.4692 - acc: 0.7735   
Epoch 9/10
2319611/2319611 [==============================] - 112s - loss: 0.4691 - acc: 0.7735   
Epoch 10/10
2319611/2319611 [==============================] - 112s - loss: 0.4687 - acc: 0.7734   


In [ ]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.user_id, un.context_type, un_data])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_flow_context50.csv", index = False)
pred.head(5)

,sample_id,is_listened
2332218,0,0.979703
2182792,1,0.583291
2092769,2,0.687123
1806368,3,0.435899
2018166,4,0.835211
